In [2]:
import torch
import pandas as pd
import numpy as np

from os.path import join

from gpytorch.kernels.rbf_kernel import RBFKernel
from gpytorch.kernels.scale_kernel import ScaleKernel

from epik.src.settings import TEST_DATA_DIR
from epik.src.kernel import SkewedVCKernel
from epik.src.model import EpiK
from scipy.stats.stats import pearsonr
from epik.src.utils import seq_to_one_hot

ModuleNotFoundError: No module named 'epik'

In [2]:
import sys

In [3]:
sys.path

['/grid/mccandlish/home_norepl/martigo/programs/epik/docs/usage',
 '/cm/local/apps/python37/lib/python37.zip',
 '/cm/local/apps/python37/lib/python3.7',
 '/cm/local/apps/python37/lib/python3.7/lib-dynload',
 '',
 '/grid/mccandlish/home_norepl/martigo/.local/lib/python3.7/site-packages',
 '/grid/mccandlish/home_norepl/martigo/.local/lib/python3.7/site-packages/gpmap_tools-0.1.0-py3.7.egg',
 '/grid/mccandlish/home_norepl/martigo/.local/lib/python3.7/site-packages/numpy-1.21.5-py3.7-linux-x86_64.egg',
 '/grid/mccandlish/home_norepl/martigo/.local/lib/python3.7/site-packages/scipy-1.7.3-py3.7-linux-x86_64.egg',
 '/grid/mccandlish/home_norepl/martigo/.local/lib/python3.7/site-packages/tqdm-4.63.0-py3.7.egg',
 '/grid/mccandlish/home_norepl/martigo/.local/lib/python3.7/site-packages/matplotlib-3.5.1-py3.7-linux-x86_64.egg',
 '/grid/mccandlish/home_norepl/martigo/.local/lib/python3.7/site-packages/seaborn-0.11.2-py3.7.egg',
 '/grid/mccandlish/home_norepl/martigo/.local/lib/python3.7/site-packa

In [3]:
n_devices = torch.cuda.device_count()
output_device = torch.device('cuda:0')
n_devices, output_device

(1, device(type='cuda', index=0))

In [4]:
def calc_L_polynomial_coeffs():
        lambdas = np.array([q**k for k in range(l+1)])
        s = l + 1
        B = np.zeros((s, s))
        idx = np.arange(s)
        for k in idx:
            k_idx = idx != k
            k_lambdas = lambdas[k_idx]
            norm_factor = 1 / np.prod(k_lambdas - lambdas[k])

            for power in idx:
                p = np.sum([np.product(v) for v in combinations(k_lambdas, l - power)])
                B[power, k] = norm_factor * (-1) ** (power) * p
        return(B)


# custom kernel for DNA sequences
class CustomKernel(gpytorch.kernels.kernel.Kernel):
    is_stationary = True
    def __init__(self, alpha, l,train_p=True, q=0.7,
                log_lda_prior=None, log_lda_constraint=None, 
                log_p_prior=None, log_p_constraint=None,
                **kwargs):
        super().__init__(**kwargs)
        
        self.odds = torch.nn.Parameter(torch.tensor([q**t/(1 - q**t) for t in range(1, l+1)]), requires_grad=False)
        self.scaling_factors = torch.tensor([(1 - q**t)**l for t in range(l+1)])
        self.scaling_factors[0] = 1
        self.scaling_factors = torch.nn.Parameter(self.scaling_factors, requires_grad=False)
        
        self.coeffs = torch.tensor(calc_L_polynomial_coeffs(), dtype=torch.float32)
        self.coeffs = torch.nn.Parameter(self.coeffs, requires_grad=False)

        # register the raw parameter
        self.register_parameter(
          name='raw_log_p', 
          parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, l, alpha), requires_grad=train_p)
        )
        self.register_parameter(
          name='raw_log_lda', 
          parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, l+1))
        )

        # set the parameter constraint to be positive, when nothing is specified
        if log_lda_constraint is None:
          log_lda_constraint = LessThan(upper_bound=0.)

        if log_p_constraint is None:
          log_p_constraint = LessThan(upper_bound=0.)

        # register the constraint
        self.register_constraint("raw_log_lda", log_lda_constraint)
        self.register_constraint("raw_log_p", log_p_constraint)

    @property
    def log_lda(self):
      return self.raw_log_lda_constraint.transform(self.raw_log_lda)

    @property
    def log_p(self):
      return self.raw_log_p_constraint.transform(self.raw_log_p)

    @log_lda.setter
    def log_lda(self, value):
      return self._set_log_lda(value)

    @log_p.setter
    def log_p(self, value):
      return self._set_log_p(value)


    def forward(self, x1, x2, diag=False, **params):
        # construct masks used for calculate rates
        masks = torch.mul(torch.unsqueeze(x1, 1), torch.unsqueeze(x2, 0))
        
        log_p = self.log_p - torch.logsumexp(self.log_p, 1)
        ps = torch.exp(log_p)
        Dpi = torch.diag(torch.exp(x2.matmul(torch.flatten(log_p)))
        
        rates = self.odds.unsqueeze(1).unsqueeze(-1)  + torch.unsqueeze(ps, 0)
        rates = rates/ps
        rates = torch.flatten(rates, start_dim=1)
        log_rates = torch.log(rates)
        
        out = torch.mul(masks.unsqueeze(2), log_rates)
        out = torch.flatten(out, start_dim=3)

        powers_nz = torch.exp(torch.sum(out, -1))
        power_0 = F.relu(torch.sum(masks, -1) - l + 1).matmul(torch.linalg.inv(Dpi))
        powers = torch.cat([power_0.unsqueeze(-1), powers_nz], dim=-1)
        powers = powers*self.scaling_factors
        
        weights = torch.matmul(self.coeffs, torch.exp(self.log_lda))
        
        k = torch.sum(torch.mul(powers, weights), -1)
                
        return k

    
class SkewVCModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, ker):
        super(SkewVCModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        base_covar_module = ker


        self.covar_module = gpytorch.kernels.MultiDeviceKernel(
            base_covar_module, device_ids=range(n_devices),
            output_device=output_device
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

SyntaxError: invalid syntax (3274207757.py, line 80)

In [5]:
# import data
alpha = 4
l = 7
q = 0.7

#!wget https://raw.githubusercontent.com/davidmccandlish/vcregression/master/vcregression/data/Smn1/smn1data.csv
  
import pandas as pd
dat = pd.read_csv("smn1data.csv", header=None)
dat = dat.rename(columns={0:"seq", 1:"psi", 2:"std", 3:"gene"})
dat['seq']=[seq[:3] + seq[4:] for seq in dat['seq']]

from collections import OrderedDict
IUPAC_VOCAB_ = OrderedDict([
    ("A", 0),
    ("U", 1),
    ("C", 2),
    ("G", 3)])

def tokenize(seq):
    return [IUPAC_VOCAB_[char] for char in seq]

seqs = [tokenize(seq) for seq in dat.seq]
seqs = torch.tensor(seqs).to(output_device)
seqs1h = torch.flatten(torch.tensor(F.one_hot(seqs),dtype=torch.float32), start_dim=1).to(output_device)
y = torch.tensor(dat.psi, dtype=torch.float32).to(output_device)


# train test data
import random
train_size = 10000
train_ids = random.sample(range(len(seqs1h)), train_size)
test_ids = random.sample(list(set(range((len(seqs1h)))).difference(train_ids)), dat.shape[0] - train_size)

train_x, test_x = seqs1h[train_ids], seqs1h[test_ids]
train_y, test_y = y[train_ids], y[test_ids]


/grid/it/data/elzar/easybuild/software/Anaconda3/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [6]:
dat.seq

0        AAAAAAA
1        AAAAAAC
2        AAAAAAG
3        AAAAAAU
4        AAAAACA
          ...   
30727    UUUUUGU
30728    UUUUUUA
30729    UUUUUUC
30730    UUUUUUG
30731    UUUUUUU
Name: seq, Length: 30732, dtype: object

In [1]:

# Define likelihood
train_noise = np.array(dat['std'].iloc[train_ids])**2
train_noise = torch.tensor(train_noise, dtype=torch.float32).to(output_device)
likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train_noise, 
                                                               learn_additional_noise=True).to(output_device)
# Define model
kernel = CustomKernel(alpha, l, train_p=True)
kernel.raw_log_lda = torch.nn.Parameter(torch.cat((torch.tensor([-100.]), 
                                                   -2*torch.arange(l))).to(output_device))
model = SkewVCModel(train_x, train_y, likelihood, kernel).to(output_device)

# predicting for test points using partitioning
checkpoint_size = train_x.shape[0]//2

test_x = test_x.cuda()
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.beta_features.checkpoint_kernel(checkpoint_size):
    f_preds = model(test_x)

NameError: name 'np' is not defined

In [44]:
class MemMonitor():
    def __init__(self):
        self.history = [0]
    
    def __call__(self):
        m = torch.cuda.memory_allocated() / 2**20
        print('Total usage {:.2f}MB; since last: {:.2f}MB'.format(m, m - self.history[0]))
        self.history.append(m)

In [45]:
monitor = MemMonitor()
monitor()

Total usage 11185.01MB; since last: 11185.01MB


In [46]:
# construct masks used for calculate rates
x1, x2 = train_x[:1000, :], train_x[:1000, :]
x1.shape, x2.shape, monitor()

Total usage 11185.01MB; since last: 11185.01MB


(torch.Size([1000, 28]), torch.Size([1000, 28]), None)

In [47]:
torch.unsqueeze(x1, 1).shape, x1.shape
monitor()

Total usage 11185.01MB; since last: 11185.01MB


In [48]:
masks[0, 0, :], x1[0, :]

(tensor([1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
         0., 1., 0., 0., 1., 0., 0., 0., 1., 0.], device='cuda:0'),
 tensor([1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
         0., 1., 0., 0., 1., 0., 0., 0., 1., 0.], device='cuda:0'))

In [49]:
masks = torch.mul(torch.unsqueeze(x1, 1), torch.unsqueeze(x2, 0))
monitor(), masks.shape

Total usage 11291.82MB; since last: 11291.82MB


(None, torch.Size([1000, 1000, 28]))

In [55]:
ps = torch.softmax(torch.rand(l, alpha).to(output_device), axis=1)
ps
monitor()

Total usage 11291.74MB; since last: 11291.74MB


In [64]:
Dpi = torch.diag(torch.exp(x2.matmul(torch.log(torch.flatten(ps)))))
monitor(), x2.shape, torch.flatten(ps).shape, log_pi.shape

Total usage 11295.77MB; since last: 11295.77MB


(None, torch.Size([1000, 28]), torch.Size([28]), torch.Size([1000]))

In [ ]:
rates = self.odds.unsqueeze(1).unsqueeze(-1)  + torch.unsqueeze(ps, 0)
rates = rates/ps
rates = torch.flatten(rates, start_dim=1)
log_rates = torch.log(rates)

out = torch.mul(masks.unsqueeze(2), log_rates)
out = torch.flatten(out, start_dim=3)

powers_nz = torch.exp(torch.sum(out, -1))
power_0 = F.relu(torch.sum(masks, -1) - l + 1).matmul(torch.linalg.inv(Dpi))
powers = torch.cat([power_0.unsqueeze(-1), powers_nz], dim=-1)
powers = powers*self.scaling_factors

weights = torch.matmul(self.coeffs, torch.exp(self.log_lda))

k = torch.sum(torch.mul(powers, weights), -1)


8940544